In [2]:
from google.colab import drive
drive.mount('/content/drive/')
!ls

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/
drive  sample_data


In [2]:
pip install tensorflow==1.8.0 tensorlayer==1.9.0 OpenEXR numpy==1.16.1 scipy==1.1.0 

     |████████████████████████████████| 49.1MB 65kB/s 
     |████████████████████████████████| 245kB 61.0MB/s 
     |████████████████████████████████| 17.3MB 211kB/s 
     |████████████████████████████████| 31.2MB 110kB/s 
     |████████████████████████████████| 3.1MB 46.2MB/s 
     |████████████████████████████████| 3.3MB 46.5MB/s 
     |████████████████████████████████| 25.4MB 127kB/s 
     |████████████████████████████████| 92kB 13.5MB/s 
     |████████████████████████████████| 51kB 9.0MB/s 
     |████████████████████████████████| 4.9MB 43.0MB/s 
     |████████████████████████████████| 12.8MB 254kB/s 
     |████████████████████████████████| 890kB 57.8MB/s 
     |████████████████████████████████| 143kB 59.7MB/s 
     |████████████████████████████████| 61kB 11.4MB/s 
  Created wheel for OpenEXR: filename=OpenEXR-1.3.2-cp36-cp36m-linux_x86_64.whl size=188411 sha256=9f64683998666099643d1aa2bec181c15a07b038d06f97ded512dab1f1532f46
  Stored in directory: /root/.cache/pip/wheels/41/06/9f/c

In [1]:
import os, sys
import tensorflow as tf
import tensorlayer as tl
import numpy as np
import network, img_io

eps = 1e-5

def print_(str, color='', bold=False):
    if color == 'w':
        sys.stdout.write('\033[93m')
    elif color == "e":
        sys.stdout.write('\033[91m')
    elif color == "m":
        sys.stdout.write('\033[95m')

    if bold:
        sys.stdout.write('\033[1m')

    sys.stdout.write(str)
    sys.stdout.write('\033[0m')
    sys.stdout.flush()


# Settings, using TensorFlow arguments
FLAGS = tf.app.flags.FLAGS
tf.app.flags.DEFINE_string('f', '', 'kernel')
tf.flags.DEFINE_integer("width", "1024", "Reconstruction image width")
tf.flags.DEFINE_integer("height", "768", "Reconstruction image height")
tf.flags.DEFINE_string("im_dir", "drive/My Drive/PBCV project/data", "Path to image directory or an individual image")
tf.flags.DEFINE_string("out_dir", "drive/My Drive/PBCV project/out", "Path to output directory")
tf.flags.DEFINE_string("params", "drive/My Drive/PBCV project/hdrcnn_params.npz", "Path to trained CNN weights")
tf.flags.DEFINE_float("scaling", "1.0", "Pre-scaling, which is followed by clipping, in order to remove compression artifacts close to highlights")
tf.flags.DEFINE_float("gamma", "1.5", "Gamma/exponential curve applied before, and inverted after, prediction. This can be used to control the boost of reconstructed pixels.")

# Round to be multiple of 32, so that autoencoder pooling+upsampling
# yields same size as input image
sx = int(np.maximum(32, np.round(FLAGS.width/32.0)*32))
sy = int(np.maximum(32, np.round(FLAGS.height/32.0)*32))
if sx != FLAGS.width or sy != FLAGS.height:
    print_("Warning: ", 'w', True)
    print_("prediction size has been changed from %dx%d pixels to %dx%d\n"%(FLAGS.width, FLAGS.height, sx, sy), 'w')
    print_("         pixels, to comply with autoencoder pooling and up-sampling.\n\n", 'w')

# Info
print_("\n\n\t-------------------------------------------------------------------\n", 'm')
print_("\t  HDR image reconstruction from a single exposure using deep CNNs\n\n", 'm')
print_("\t  Prediction settings\n", 'm')
print_("\t  -------------------\n", 'm')
print_("\t  Input image directory/file:     %s\n" % FLAGS.im_dir, 'm')
print_("\t  Output directory:               %s\n" % FLAGS.out_dir, 'm')
print_("\t  CNN weights:                    %s\n" % FLAGS.params, 'm')
print_("\t  Prediction resolution:          %dx%d pixels\n" % (sx, sy), 'm')
if FLAGS.scaling > 1.0:
    print_("\t  Pre-scaling:                    %0.4f\n" % FLAGS.scaling, 'm')
if FLAGS.gamma > 1.0 + eps or FLAGS.gamma < 1.0 - eps:
    print_("\t  Gamma:                          %0.4f\n" % FLAGS.gamma, 'm')
print_("\t-------------------------------------------------------------------\n\n\n", 'm')

# Single frame
frames = [FLAGS.im_dir]

# If directory is supplied, get names of all files in the path
if os.path.isdir(FLAGS.im_dir):
    frames = [os.path.join(FLAGS.im_dir, name)
              for name in sorted(os.listdir(FLAGS.im_dir))
              if os.path.isfile(os.path.join(FLAGS.im_dir, name))]

# Placeholder for image input
x = tf.placeholder(tf.float32, shape=[1, sy, sx, 3])

# HDR reconstruction autoencoder model
print_("Network setup:\n")
net = network.model(x)

# The CNN prediction (this also includes blending with input image x)
y = network.get_final(net, x)



	-------------------------------------------------------------------
	  HDR image reconstruction from a single exposure using deep CNNs

	  Prediction settings
	  -------------------
	  Input image directory/file:     drive/My Drive/PBCV project/data
	  Output directory:               drive/My Drive/PBCV project/out
	  CNN weights:                    drive/My Drive/PBCV project/hdrcnn_params.npz
	  Prediction resolution:          1024x768 pixels
	  Gamma:                          1.5000
	-------------------------------------------------------------------


Network setup:
[TL] InputLayer  input_layer: (1, 768, 1024, 3)
[TL] InputLayer  encoder/input_layer_bgr: (1, 768, 1024, 3)
[TL] Conv2dLayer encoder/h1/conv_1: shape: [3, 3, 3, 64] strides: [1, 1, 1, 1] pad: SAME act: relu
[TL] Conv2dLayer encoder/h1/conv_2: shape: [3, 3, 64, 64] strides: [1, 1, 1, 1] pad: SAME act: relu
[TL] PoolLayer encoder/h1/pool: ksize: [1, 2, 2, 1] strides: [1, 2, 2, 1] padding: SAME pool: max_pool
[TL] Conv2

In [0]:
# Camera 8
# TensorFlow session for running inference
sess = tf.InteractiveSession()

# Load trained CNN weights
print_("\nLoading trained parameters from '%s'..."%FLAGS.params)
load_params = tl.files.load_npz(name=FLAGS.params)
tl.files.assign_params(sess, load_params, net)
print_("\tdone\n")

if not os.path.exists(FLAGS.out_dir):
    os.makedirs(FLAGS.out_dir)

print_("\nStarting prediction...\n\n")
k = 0
print(len(frames))
for i in range(len(frames)):
    print("Frame %d: '%s'"%(i,frames[i]))

    try:
        # Read frame
        print_("\tReading...")
        x_buffer = img_io.readLDR(frames[i], (sy,sx), True, FLAGS.scaling)
        print_("\tdone")

        print_("\t(Saturation: %0.2f%%)\n" % (100.0*(x_buffer>=1).sum()/x_buffer.size), 'm')

        # Run prediction.
        # The gamma value is used to allow for boosting/reducing the intensity of
        # the reconstructed highlights. If y = f(x) is the reconstruction, the gamma
        # g alters this according to y = f(x^(1/g))^g
        print_("\tInference...")
        feed_dict = {x: np.power(np.maximum(x_buffer, 0.0), 1.0/FLAGS.gamma)}
        y_predict = sess.run([y], feed_dict=feed_dict)
        y_predict = np.power(np.maximum(y_predict, 0.0), FLAGS.gamma)
        print_("\tdone\n")


        # Gamma corrected output
        y_gamma = np.power(np.maximum(y_predict, 0.0), 0.5)

        # Write to disc
        print_("\tWriting...")
        k += 1;
        img_io.writeLDR(x_buffer, '%s/%06d_in.png' % (FLAGS.out_dir, k), -3)
        img_io.writeLDR(y_gamma, '%s/%06d_out.png' % (FLAGS.out_dir, k), -3)
        img_io.writeEXR(y_predict, '%s/%06d_out.exr' % (FLAGS.out_dir, k))
        print_("\tdone\n")

    except img_io.IOException as e:
        print_("\n\t\tWarning! ", 'w', True)
        print_("%s\n"%e, 'w')
    except Exception as e:    
        print_("\n\t\tError: ", 'e', True)
        print_("%s\n"%e, 'e')


print_("Done!\n")

sess.close()


Loading trained parameters from 'drive/My Drive/PBCV project/hdrcnn_params.npz'...	done

Starting prediction...

106
Frame 0: 'drive/My Drive/PBCV project/data/Camera8_1.png'
	Reading...	done	(Saturation: 17.63%)
	Inference...	done
	Writing...	done
Frame 1: 'drive/My Drive/PBCV project/data/Camera8_10.png'
	Reading...	done	(Saturation: 17.64%)
	Inference...	done
	Writing...	done
Frame 2: 'drive/My Drive/PBCV project/data/Camera8_11.png'
	Reading...	done	(Saturation: 17.60%)
	Inference...	done
	Writing...	done
Frame 3: 'drive/My Drive/PBCV project/data/Camera8_12.png'
	Reading...	done	(Saturation: 17.65%)
	Inference...	done
	Writing...	done
Frame 4: 'drive/My Drive/PBCV project/data/Camera8_13.png'
	Reading...	done	(Saturation: 17.80%)
	Inference...	done
	Writing...	done
Frame 5: 'drive/My Drive/PBCV project/data/Camera8_14.png'
	Reading...	done	(Saturation: 17.95%)
	Inference...	done
	Writing...	done
Frame 6: 'drive/My Drive/PBCV project/data/Camera8_15.png'
	Reading...	done	(Saturati

In [0]:
# Camera 6
# TensorFlow session for running inference
sess = tf.InteractiveSession()

# Load trained CNN weights
print_("\nLoading trained parameters from '%s'..."%FLAGS.params)
load_params = tl.files.load_npz(name=FLAGS.params)
tl.files.assign_params(sess, load_params, net)
print_("\tdone\n")

if not os.path.exists(FLAGS.out_dir):
    os.makedirs(FLAGS.out_dir)

print_("\nStarting prediction...\n\n")
k = 0
print(len(frames))
for i in range(len(frames)):
    print("Frame %d: '%s'"%(i,frames[i]))

    try:
        # Read frame
        print_("\tReading...")
        x_buffer = img_io.readLDR(frames[i], (sy,sx), True, FLAGS.scaling)
        print_("\tdone")
        
        print_("\t(Saturation: %0.2f%%)\n" % (100.0*(x_buffer>=1).sum()/x_buffer.size), 'm')

        # Run prediction.
        # The gamma value is used to allow for boosting/reducing the intensity of
        # the reconstructed highlights. If y = f(x) is the reconstruction, the gamma
        # g alters this according to y = f(x^(1/g))^g
        print_("\tInference...")
        feed_dict = {x: np.power(np.maximum(x_buffer, 0.0), 1.0/FLAGS.gamma)}
        y_predict = sess.run([y], feed_dict=feed_dict)
        y_predict = np.power(np.maximum(y_predict, 0.0), FLAGS.gamma)
        print_("\tdone\n")


        # Gamma corrected output
        y_gamma = np.power(np.maximum(y_predict, 0.0), 0.5)

        # Write to disc
        print_("\tWriting...")
        k += 1;
        img_io.writeLDR(x_buffer, '%s/%06d_in.png' % (FLAGS.out_dir, k), -3)
        img_io.writeLDR(y_gamma, '%s/%06d_out.png' % (FLAGS.out_dir, k), -3)
        img_io.writeEXR(y_predict, '%s/%06d_out.exr' % (FLAGS.out_dir, k))
        print_("\tdone\n")

    except img_io.IOException as e:
        print_("\n\t\tWarning! ", 'w', True)
        print_("%s\n"%e, 'w')
    except Exception as e:    
        print_("\n\t\tError: ", 'e', True)
        print_("%s\n"%e, 'e')


print_("Done!\n")

sess.close()


Loading trained parameters from 'drive/My Drive/PBCV project/hdrcnn_params.npz'...	done

Starting prediction...

99
Frame 0: 'drive/My Drive/PBCV project/data/Camera6_1.png'
	Reading...	done	(Saturation: 6.85%)
	Inference...	done
	Writing...	done
Frame 1: 'drive/My Drive/PBCV project/data/Camera6_10.png'
	Reading...	done	(Saturation: 7.17%)
	Inference...	done
	Writing...	done
Frame 2: 'drive/My Drive/PBCV project/data/Camera6_11.png'
	Reading...	done	(Saturation: 7.18%)
	Inference...	done
	Writing...	done
Frame 3: 'drive/My Drive/PBCV project/data/Camera6_12.png'
	Reading...	done	(Saturation: 7.12%)
	Inference...	done
	Writing...	done
Frame 4: 'drive/My Drive/PBCV project/data/Camera6_13.png'
	Reading...	done	(Saturation: 7.27%)
	Inference...	done
	Writing...	done
Frame 5: 'drive/My Drive/PBCV project/data/Camera6_14.png'
	Reading...	done	(Saturation: 7.35%)
	Inference...	done
	Writing...	done
Frame 6: 'drive/My Drive/PBCV project/data/Camera6_15.png'
	Reading...	done	(Saturation: 7.2

In [2]:
# Camera 4
# TensorFlow session for running inference
sess = tf.InteractiveSession()

# Load trained CNN weights
print_("\nLoading trained parameters from '%s'..."%FLAGS.params)
load_params = tl.files.load_npz(name=FLAGS.params)
tl.files.assign_params(sess, load_params, net)
print_("\tdone\n")

if not os.path.exists(FLAGS.out_dir):
    os.makedirs(FLAGS.out_dir)

print_("\nStarting prediction...\n\n")
k = 0
print(len(frames))
for i in range(len(frames)):
    print("Frame %d: '%s'"%(i,frames[i]))

    try:
        # Read frame
        print_("\tReading...")
        x_buffer = img_io.readLDR(frames[i], (sy,sx), True, FLAGS.scaling)
        print_("\tdone")

        print_("\t(Saturation: %0.2f%%)\n" % (100.0*(x_buffer>=1).sum()/x_buffer.size), 'm')

        # Run prediction.
        # The gamma value is used to allow for boosting/reducing the intensity of
        # the reconstructed highlights. If y = f(x) is the reconstruction, the gamma
        # g alters this according to y = f(x^(1/g))^g
        print_("\tInference...")
        feed_dict = {x: np.power(np.maximum(x_buffer, 0.0), 1.0/FLAGS.gamma)}
        y_predict = sess.run([y], feed_dict=feed_dict)
        y_predict = np.power(np.maximum(y_predict, 0.0), FLAGS.gamma)
        print_("\tdone\n")


        # Gamma corrected output
        y_gamma = np.power(np.maximum(y_predict, 0.0), 0.5)

        # Write to disc
        print_("\tWriting...")
        k += 1;
        img_io.writeLDR(x_buffer, '%s/%06d_in.png' % (FLAGS.out_dir, k), -3)
        img_io.writeLDR(y_gamma, '%s/%06d_out.png' % (FLAGS.out_dir, k), -3)
        img_io.writeEXR(y_predict, '%s/%06d_out.exr' % (FLAGS.out_dir, k))
        print_("\tdone\n")

    except img_io.IOException as e:
        print_("\n\t\tWarning! ", 'w', True)
        print_("%s\n"%e, 'w')
    except Exception as e:    
        print_("\n\t\tError: ", 'e', True)
        print_("%s\n"%e, 'e')


print_("Done!\n")

sess.close()


Loading trained parameters from 'drive/My Drive/PBCV project/hdrcnn_params.npz'...	done

Starting prediction...

99
Frame 0: 'drive/My Drive/PBCV project/data/Camera4_1.png'
	Reading...	done	(Saturation: 2.29%)
	Inference...	done
	Writing...	done
Frame 1: 'drive/My Drive/PBCV project/data/Camera4_10.png'
	Reading...	done	(Saturation: 2.43%)
	Inference...	done
	Writing...	done
Frame 2: 'drive/My Drive/PBCV project/data/Camera4_11.png'
	Reading...	done	(Saturation: 2.41%)
	Inference...	done
	Writing...	done
Frame 3: 'drive/My Drive/PBCV project/data/Camera4_12.png'
	Reading...	done	(Saturation: 2.39%)
	Inference...	done
	Writing...	done
Frame 4: 'drive/My Drive/PBCV project/data/Camera4_13.png'
	Reading...	done	(Saturation: 2.43%)
	Inference...	done
	Writing...	done
Frame 5: 'drive/My Drive/PBCV project/data/Camera4_14.png'
	Reading...	done	(Saturation: 2.46%)
	Inference...	done
	Writing...	done
Frame 6: 'drive/My Drive/PBCV project/data/Camera4_15.png'
	Reading...	done	(Saturation: 2.4

In [3]:
import cv2 ,os, glob, pickle, shutil
files_loc = 'drive/My Drive/PBCV project/out/*_in.png'
files = glob.glob(files_loc,recursive = True)
for i in range(0,len(files)):
  img = cv2.imread(files[i])
  file_name = os.path.basename(files[i])
  out_name = 'drive/My Drive/PBCV project/Camera4_in/' + file_name
  print(out_name)
  cv2.imwrite(out_name,img)


drive/My Drive/PBCV project/Camera4_in/000001_in.png
drive/My Drive/PBCV project/Camera4_in/000002_in.png
drive/My Drive/PBCV project/Camera4_in/000003_in.png
drive/My Drive/PBCV project/Camera4_in/000004_in.png
drive/My Drive/PBCV project/Camera4_in/000005_in.png
drive/My Drive/PBCV project/Camera4_in/000006_in.png
drive/My Drive/PBCV project/Camera4_in/000007_in.png
drive/My Drive/PBCV project/Camera4_in/000008_in.png
drive/My Drive/PBCV project/Camera4_in/000009_in.png
drive/My Drive/PBCV project/Camera4_in/000010_in.png
drive/My Drive/PBCV project/Camera4_in/000011_in.png
drive/My Drive/PBCV project/Camera4_in/000012_in.png
drive/My Drive/PBCV project/Camera4_in/000013_in.png
drive/My Drive/PBCV project/Camera4_in/000014_in.png
drive/My Drive/PBCV project/Camera4_in/000015_in.png
drive/My Drive/PBCV project/Camera4_in/000016_in.png
drive/My Drive/PBCV project/Camera4_in/000017_in.png
drive/My Drive/PBCV project/Camera4_in/000018_in.png
drive/My Drive/PBCV project/Camera4_in/000019_